In [1]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array
import string
import re
from numpy import array, argmax, random, take
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras_preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', 200)
from pickle import load
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.layers import TimeDistributed


In [13]:
import collections
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from tensorflow.keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy, huber_loss, categorical_crossentropy

In [3]:
import os
def load_data(path):
    """
    Load Dataset from File
    """
    input_file = os.path.join(path)
    with open(input_file, "r",encoding="utf8") as f:
        data = f.read().splitlines()

    return data


In [4]:
english_sentences = load_data('tal_eng.txt')
tamil_sentences = load_data('tal_tal.txt')
print('Dataset Loaded')

Dataset Loaded


In [5]:
type(english_sentences)

list

In [6]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, tamil_sentences[sample_i]))

small_vocab_en Line 1:  joker
small_vocab_fr Line 1:  கோமாளித்தனம்
small_vocab_en Line 2:  Related Pages
small_vocab_fr Line 2:  தொடர்பான பக்கங்கள்


In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
tamil_words_counter = collections.Counter([word for sentence in tamil_sentences for word in sentence.split()])
print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} tamil words.'.format(len([word for sentence in tamil_sentences for word in sentence.split()])))
print('{} unique tamil words.'.format(len(tamil_words_counter)))
print('10 Most common words in the tamil dataset:')
print('"' + '" "'.join(list(zip(*tamil_words_counter.most_common(10)))[0]) + '"')

12135 English words.
4649 unique English words.
10 Most common words in the English dataset:
"the" "in" "of" "and" "to" "is" "was" "In" "a" "people"

11335 tamil words.
5886 unique tamil words.
10 Most common words in the tamil dataset:
"-" "பேர்" "சோவியத்" "போர்" "ஒரு" "பல" "நாடுகள்" "நேச" "மே" "ஜூன்"


In [8]:
def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [9]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')
# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [10]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
# Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    return preprocess_x, preprocess_y, x_tk, y_tk
preproc_english_sentences, preproc_tamil_sentences, english_tokenizer, tamil_tokenizer =\
    preprocess(english_sentences, tamil_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_tamil_sequence_length = preproc_tamil_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
tamil_vocab_size = len(tamil_tokenizer.word_index)
print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max tamil sentence length:", max_tamil_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("Tamil vocabulary size:", tamil_vocab_size)

Data Preprocessed
Max English sentence length: 82
Max tamil sentence length: 59
English vocabulary size: 3375
Tamil vocabulary size: 5650


In [11]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
print('`logits_to_text` function loaded.')


`logits_to_text` function loaded.


In [15]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, tamil_vocab_size):
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(4720))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss = huber_loss, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model
tmp_x = pad(preproc_english_sentences, max_tamil_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_tamil_sentences.shape[-2], 1))
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_tamil_sequence_length,
    english_vocab_size,
    tamil_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_tamil_sentences, batch_size=128, epochs=10, validation_split=0.2)
# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], tamil_tokenizer))

Epoch 1/10
9/9 [==============================] - 11s 1s/step - loss: 206.5008 - accuracy: 0.8300 - val_loss: 337.0925 - val_accuracy: 0.8266
Epoch 2/10
9/9 [==============================] - 12s 1s/step - loss: 206.5008 - accuracy: 0.8300 - val_loss: 337.0925 - val_accuracy: 0.8266
Epoch 3/10
9/9 [==============================] - 12s 1s/step - loss: 206.5008 - accuracy: 0.8300 - val_loss: 337.0925 - val_accuracy: 0.8266
Epoch 4/10
9/9 [==============================] - 13s 1s/step - loss: 206.5008 - accuracy: 0.8300 - val_loss: 337.0925 - val_accuracy: 0.8266
Epoch 5/10
9/9 [==============================] - 15s 2s/step - loss: 206.5008 - accuracy: 0.8300 - val_loss: 337.0925 - val_accuracy: 0.8266
Epoch 6/10
9/9 [==============================] - 14s 2s/step - loss: 206.5008 - accuracy: 0.8300 - val_loss: 337.0925 - val_accuracy: 0.8266
Epoch 7/10
9/9 [==============================] - 11s 1s/step - loss: 206.5008 - accuracy: 0.8300 - val_loss: 337.0925 - val_accuracy: 0.8266
Epoch 

In [16]:
from keras.models import Sequential
def embed_model(input_shape, output_sequence_length, english_vocab_size, tamil_vocab_size):
    learning_rate = 1e-3
    rnn = GRU(64, return_sequences=True, activation="tanh")
    
    embedding = Embedding(tamil_vocab_size, 64, input_length=input_shape[1]) 
    logits = TimeDistributed(Dense(tamil_vocab_size, activation="softmax"))
    
    model = Sequential()
    #em can only be used in first layer --> Keras Documentation
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=huber_loss,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tmp_x = pad(preproc_english_sentences, max_tamil_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_tamil_sentences.shape[-2]))
embeded_model = embed_model(
    tmp_x.shape,
    max_tamil_sequence_length,
    english_vocab_size,
    tamil_vocab_size)
embeded_model.fit(tmp_x, preproc_tamil_sentences, batch_size=1024, epochs=10, validation_split=0.2)
print(logits_to_text(embeded_model.predict(tmp_x[:1])[0], tamil_tokenizer))

Epoch 1/10
2/2 [==============================] - 67s 2s/step - loss: 206.5008 - accuracy: 0.0000e+00 - val_loss: 337.0925 - val_accuracy: 0.0000e+00
Epoch 2/10
2/2 [==============================] - 54s 1s/step - loss: 206.5008 - accuracy: 0.0000e+00 - val_loss: 337.0925 - val_accuracy: 0.0000e+00
Epoch 3/10
2/2 [==============================] - 58s 1s/step - loss: 206.5008 - accuracy: 0.0000e+00 - val_loss: 337.0925 - val_accuracy: 0.0000e+00
Epoch 4/10
2/2 [==============================] - 77s 2s/step - loss: 206.5008 - accuracy: 0.0000e+00 - val_loss: 337.0925 - val_accuracy: 0.0000e+00
Epoch 5/10
2/2 [==============================] - 54s 1s/step - loss: 206.5008 - accuracy: 0.0000e+00 - val_loss: 337.0925 - val_accuracy: 0.0000e+00
Epoch 6/10
2/2 [==============================] - 73s 2s/step - loss: 206.5008 - accuracy: 0.0000e+00 - val_loss: 337.0925 - val_accuracy: 0.0000e+00
Epoch 7/10
2/2 [==============================] - 62s 2s/step - loss: 206.5008 - accuracy: 0.0000e+0

In [17]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, tamil_vocab_size):
   
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences = True, dropout = 0.1), 
                           input_shape = input_shape[1:]))
    model.add(TimeDistributed(Dense(tamil_vocab_size, activation = 'softmax')))
    model.compile(loss = huber_loss, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model
tmp_x = pad(preproc_english_sentences, preproc_tamil_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_tamil_sentences.shape[-2], 1))
bidi_model = bd_model(
    tmp_x.shape,
    preproc_tamil_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(tamil_tokenizer.word_index)+1)
bidi_model.fit(tmp_x, preproc_tamil_sentences, batch_size=1024, epochs=20, validation_split=0.2)
# Print prediction(s)
print(logits_to_text(bidi_model.predict(tmp_x[:1])[0], tamil_tokenizer))

Epoch 1/20
2/2 [==============================] - 41s 2s/step - loss: 206.5008 - accuracy: 0.8304 - val_loss: 337.0925 - val_accuracy: 0.8268
Epoch 2/20
2/2 [==============================] - 29s 2s/step - loss: 206.5008 - accuracy: 0.8305 - val_loss: 337.0925 - val_accuracy: 0.8268
Epoch 3/20
2/2 [==============================] - 28s 2s/step - loss: 206.5008 - accuracy: 0.8310 - val_loss: 337.0925 - val_accuracy: 0.8268
Epoch 4/20
2/2 [==============================] - 33s 2s/step - loss: 206.5008 - accuracy: 0.8307 - val_loss: 337.0925 - val_accuracy: 0.8268
Epoch 5/20
2/2 [==============================] - 33s 2s/step - loss: 206.5008 - accuracy: 0.8306 - val_loss: 337.0925 - val_accuracy: 0.8268
Epoch 6/20
2/2 [==============================] - 35s 2s/step - loss: 206.5008 - accuracy: 0.8306 - val_loss: 337.0925 - val_accuracy: 0.8268
Epoch 7/20
2/2 [==============================] - 27s 2s/step - loss: 206.5008 - accuracy: 0.8308 - val_loss: 337.0925 - val_accuracy: 0.8268
Epoch 

In [18]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, tamil_vocab_size):
  
    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(128, return_sequences = True))
    model.add(TimeDistributed(Dense(tamil_vocab_size, activation = 'softmax')))
    
    model.compile(loss = huber_loss, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model
tmp_x = pad(preproc_english_sentences)
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))
encodeco_model = encdec_model(
    tmp_x.shape,
    preproc_tamil_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(tamil_tokenizer.word_index)+1)
encodeco_model.fit(tmp_x, preproc_tamil_sentences, batch_size=1024, epochs=20, validation_split=0.2)
print(logits_to_text(encodeco_model.predict(tmp_x[:1])[0], tamil_tokenizer))

Epoch 1/20
2/2 [==============================] - 43s 3s/step - loss: 206.5008 - accuracy: 0.8519 - val_loss: 337.0925 - val_accuracy: 0.8451
Epoch 2/20
2/2 [==============================] - 36s 2s/step - loss: 206.5008 - accuracy: 0.8519 - val_loss: 337.0925 - val_accuracy: 0.8451
Epoch 3/20
2/2 [==============================] - 36s 2s/step - loss: 206.5008 - accuracy: 0.8519 - val_loss: 337.0925 - val_accuracy: 0.8451
Epoch 4/20
2/2 [==============================] - 33s 2s/step - loss: 206.5008 - accuracy: 0.8519 - val_loss: 337.0925 - val_accuracy: 0.8451
Epoch 5/20
2/2 [==============================] - 41s 2s/step - loss: 206.5008 - accuracy: 0.8519 - val_loss: 337.0925 - val_accuracy: 0.8451
Epoch 6/20
2/2 [==============================] - 39s 2s/step - loss: 206.5008 - accuracy: 0.8519 - val_loss: 337.0925 - val_accuracy: 0.8451
Epoch 7/20
2/2 [==============================] - 36s 2s/step - loss: 206.5008 - accuracy: 0.8519 - val_loss: 337.0925 - val_accuracy: 0.8451
Epoch 

In [19]:
def model_final(input_shape, output_sequence_length, english_vocab_size, tamil_vocab_size):
  
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(tamil_vocab_size,activation='softmax')))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model
print('Final Model Loaded')

Final Model Loaded


In [20]:
def final_predictions(x, y, x_tk, y_tk):
    global predictions
    tmp_X = pad(preproc_english_sentences)
    model = model_final(tmp_X.shape,
                        preproc_tamil_sentences.shape[1],
                        len(english_tokenizer.word_index)+1,
                        len(tamil_tokenizer.word_index)+1)
    
    model.fit(tmp_X, preproc_tamil_sentences, batch_size = 1024, epochs = 17, validation_split = 0.2)
 
    
final_predictions(preproc_english_sentences, preproc_tamil_sentences, english_tokenizer, tamil_tokenizer)

Epoch 1/17
2/2 [==============================] - 92s 5s/step - loss: 8.6338 - accuracy: 0.0073 - val_loss: 2.8550 - val_accuracy: 0.8492
Epoch 2/17
2/2 [==============================] - 141s 4s/step - loss: 2.8117 - accuracy: 0.8540 - val_loss: 2.0450 - val_accuracy: 0.8492
Epoch 3/17
2/2 [==============================] - 84s 3s/step - loss: 1.9537 - accuracy: 0.8540 - val_loss: 1.6771 - val_accuracy: 0.8492
Epoch 4/17
2/2 [==============================] - 92s 3s/step - loss: 1.5628 - accuracy: 0.8540 - val_loss: 1.6554 - val_accuracy: 0.8492
Epoch 5/17
2/2 [==============================] - 76s 3s/step - loss: 1.4952 - accuracy: 0.8540 - val_loss: 1.8522 - val_accuracy: 0.8492
Epoch 6/17
2/2 [==============================] - 70s 3s/step - loss: 1.6432 - accuracy: 0.8540 - val_loss: 1.7666 - val_accuracy: 0.8492
Epoch 7/17
2/2 [==============================] - 70s 3s/step - loss: 1.5170 - accuracy: 0.8540 - val_loss: 1.8358 - val_accuracy: 0.8492
Epoch 8/17
2/2 [=================